In [1]:
# импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns             
plt.style.use('ggplot')     

%matplotlib inline

In [18]:
# загрузка файлов
ratings = pd.read_csv("ratings.csv")
tags= pd.read_csv("tags.csv")
movies = pd.read_csv("movies.csv")

In [19]:
ratings[['userId', 'movieId', 'rating']]

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [20]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [21]:
movies = movies.merge(ratings, on = 'movieId', how = 'inner')
movies

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483
...,...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184,4.0,1537109082
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184,3.5,1537109545
100833,193585,Flint (2017),Drama,184,3.5,1537109805
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184,3.5,1537110021


In [24]:
df = movies.groupby(by=['userId','movieId'], as_index=False, sort=False).mean()
df = df[['userId','movieId','rating']]
df

,userId,movieId,rating
0,1,1,4.0
1,5,1,4.0
2,7,1,4.5
3,15,1,2.5
4,17,1,4.5
...,...,...,...
100831,184,193581,4.0
100832,184,193583,3.5
100833,184,193585,3.5
100834,184,193587,3.5


In [26]:
#посмотрим, есть ли пропуски
df.isnull().sum()

userId     0
movieId    0
rating     0
dtype: int64

In [29]:
Ratings = ratings.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
Ratings.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
R = Ratings.values
user_ratings_mean = np.mean(R, axis = 1)
Ratings_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [35]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(Ratings_demeaned, k = 50)

In [36]:
sigma = np.diag(sigma)

In [37]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [39]:
preds = pd.DataFrame(all_user_predicted_ratings, columns = Ratings.columns)
preds

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
0,2.167328,0.402751,0.840184,-0.076281,-0.551337,2.504091,-0.890114,-0.026443,0.196974,1.593259,...,-0.023453,-0.019967,-0.026939,-0.026939,-0.023453,-0.026939,-0.023453,-0.023453,-0.023453,-0.058732
1,0.211459,0.006658,0.033455,0.017419,0.183430,-0.062473,0.083037,0.024158,0.049330,-0.152530,...,0.019498,0.016777,0.022219,0.022219,0.019498,0.022219,0.019498,0.019498,0.019498,0.032281
2,0.003588,0.030518,0.046393,0.008176,-0.006247,0.107328,-0.012416,0.003779,0.007297,-0.059362,...,0.005909,0.006209,0.005610,0.005610,0.005909,0.005610,0.005909,0.005909,0.005909,0.008004
3,2.051549,-0.387104,-0.252199,0.087562,0.130465,0.270210,0.477835,0.040313,0.025858,-0.017365,...,0.004836,0.004172,0.005500,0.005500,0.004836,0.005500,0.004836,0.004836,0.004836,-0.023311
4,1.344738,0.778511,0.065749,0.111744,0.273144,0.584426,0.254930,0.128788,-0.085541,1.023455,...,-0.008042,-0.007419,-0.008664,-0.008664,-0.008042,-0.008664,-0.008042,-0.008042,-0.008042,-0.010127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,2.501444,-0.139015,-0.082080,0.079300,-0.158770,-0.587275,2.368039,-0.014790,-0.093695,-0.182211,...,-0.038935,-0.032500,-0.045369,-0.045369,-0.038935,-0.045369,-0.038935,-0.038935,-0.038935,-0.045416
606,2.849138,1.368651,0.341869,0.000534,-0.272603,1.529573,-0.078889,-0.013913,0.075251,1.398731,...,0.006789,0.006108,0.007471,0.007471,0.006789,0.007471,0.006789,0.006789,0.006789,-0.030371
607,2.161364,2.670920,2.128460,0.036007,0.128314,3.684387,-0.028717,0.195936,0.073591,2.406330,...,-0.044003,-0.041123,-0.046882,-0.046882,-0.044003,-0.046882,-0.044003,-0.044003,-0.044003,0.071872
608,0.780206,0.533649,0.096454,0.029945,0.087756,0.209604,0.078704,0.061626,-0.064337,1.242835,...,0.004355,0.004268,0.004442,0.004442,0.004355,0.004442,0.004355,0.004355,0.004355,-0.001346


переходим непоосредственно к модели:

In [40]:
from sklearn.model_selection import train_test_split 

# делим в пропорции 80 к 20
df_train, df_test = train_test_split(Ratings, test_size = 0.2, random_state=42)

In [42]:
y_train = df_train['rating'].to_numpy()
y_test = df_test['rating'].to_numpy()

X_train = df_train.drop('rating', axis=1).to_numpy()
X_test = df_test.drop('rating', axis=1).to_numpy()

feature_names = df_train.drop('rating', axis=1).columns

In [43]:
y_train

array([2.5, 5. , 5. , ..., 4. , 4. , 3. ])

In [45]:
X_train

array([[  306, 35836],
       [  182,  3000],
       [   40,   308],
       ...,
       [  387,  8670],
       [    6,    15],
       [  386,   589]])

In [52]:
y_mean = np.mean(y_train)                   
y_pred_naive = np.ones(len(y_test)) * y_mean

In [53]:
#подгрузим метрики
from sklearn import metrics

In [54]:
def print_metrics(y_test,y_pred_naive):
    print('MAE:', "%.4f" % metrics.mean_absolute_error(y_test, y_pred_naive))
    print('RMSE:', "%.4f" % np.sqrt(metrics.mean_squared_error(y_test, y_pred_naive)))
    pass

print_metrics(y_test,y_pred_naive)

MAE: 0.8318
RMSE: 1.0476


In [55]:
#наивный прогноз показывает достаточно высокую ошибку, поэтому попробуем линейную регрессию
from sklearn.linear_model import LinearRegression 
model_regression = LinearRegression()
model_regression.fit(X_train, y_train)
y_pred_regr = model_regression.predict(X_test)

In [56]:
print_metrics(y_test, y_pred_regr)

MAE: 0.8353
RMSE: 1.0466


In [63]:
#Кросс-валидация
y = df['rating'].to_numpy()
X = df.drop('rating', axis=1).to_numpy()

In [59]:
from sklearn.model_selection import cross_val_score

model = LinearRegression()

cv_linear_reg_results = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
cv_linear_reg_results

array([-1.10168484, -1.1146018 , -1.14251826, -1.04054811, -1.24594351])

In [60]:
np.mean(np.sqrt(-1*cv_linear_reg_results))

1.0621072491263033

качество регрессии так себе, но других идей у нас нет